In [ ]:
"""
実行結果を解析するスクリプトです。
"""
import pandas as pd

data = pd.read_csv("../data/exchange_data/step3_glove.csv")

data = data[~data["result"].str.contains("empty|keyerror")]
data.to_csv("../data/exchange_data/step3_result.csv")
print(len(data))

In [24]:
"""
コーパスの確認をするスクリプトです。
"""
import re
import pandas as pd

clean_text = re.compile('[!"#$%&\'\\\\()*+,-./:;<=>?@[\\]^_`{|}~「」〔〕“”〈〉『』【】＆＊・（）＄＃＠。、？！｀＋￥％]')

data = pd.read_csv("../data/train_data/recipe_memo.csv",names=["step"])
cleaned_data = data.apply(lambda data:data.str.replace(clean_text,'',regex=True))

In [25]:
cleaned_data = cleaned_data.dropna(subset=["step"])
cleaned_data["str_len"] = cleaned_data["step"].apply(lambda data:len(data))
output = cleaned_data[cleaned_data["str_len"] != 0]
output.head(10)

,step,str_len
0,鍋に豚肉をいれて酒砂糖みりん醤油しょうが薄切りにんにくで煮込みます,33
1,にんじんとれんこんをおろし金でおろします,20
2,挽肉と玉ねぎのみじん切りを加えよく,17
4,混ぜます,4
5,塩コショウを加えピーマンに詰めあとは焼くだけ,22
6,少し蒸らして火を通しできあがり,15
7,たれはおろしだれが一番,11
8,米は研がずにそのまま使っちゃえ,15
10,なんてったって簡単チーズリゾット,16
11,水も適当に普通炊くよりも多めです,16


In [27]:
output.describe()
output["step"].to_csv('../data/train_data/fix_recipe_memo.csv')

In [29]:
output["str_len"].sum()

305153110

In [34]:
"""
コーパスの作成
"""
from tqdm import tqdm as tq
import MeCab

m = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd -Owakati")

# 合計語彙数
num = 0
# データの総文字数
str_len = output["str_len"].sum()
# 学習データ
corpas = ""

for step in tq(output["step"],total=len(output)):
    wakati = m.parse(step)
    num += len(wakati)
    tmp = "".join(wakati)
    corpas += tmp + "\n"

with open("../data/train_data/cookpad_step_corpas.txt",'w') as out_corpas:
    out_corpas.write(corpas)

print("平均語彙サイズ:",str_len/num)
print("平均語彙数:",num/len(output))

100%|██████████| 10811257/10811257 [05:07<00:00, 35167.45it/s]


平均語彙サイズ: 0.6433526802049334
平均語彙数: 43.87251130927699
